In [1]:
! pip install dotenv

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
    ERROR: Command errored out with exit status 1:
     command: /Users/alexandermcmullan/opt/anaconda3/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/br/s09sd3jx7ws6f36s7rlyjk740000gp/T/pip-install-mchoz7ei/dotenv_8f443cd0515340cabf7cb6f9769dd643/setup.py'"'"'; __file__='"'"'/private/var/folders/br/s09sd3jx7ws6f36s7rlyjk740000gp/T/pip-install-mchoz7ei/dotenv_8f443cd0515340cabf7cb6f9769dd643/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/br/s09sd3jx7ws6f36s7rlyjk740000gp/T/pip-pip-egg-info-_n_ii6hc
         cwd: /private/var/folders/br/s09sd3jx7ws6f36s7rlyjk740000gp/T/pip-install-mchoz7ei/dotenv_8f443cd0515340cabf7cb6f9769dd643/
    Complete output (1589 lines):
        ERROR: Command errored out with exit status 1:
         command: /

In [2]:
! pip install python-dotenv

  Using cached python_dotenv-0.19.1-py2.py3-none-any.whl (17 kB)


In [9]:
import subprocess
import json
import os

from constants import BTC, BTCTEST, ETH
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware

# connect Web3
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))

# enable PoA middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# set gas price strategy to built-in "medium" algorithm (est ~5min per tx)
# see https://web3py.readthedocs.io/en/stable/gas_price.html?highlight=gas
# see https://ethgasstation.info/ API for a more accurate strategy
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

# including a mnemonic with prefunded test tokens for testing
mnemonic = os.getenv("mnemonic")

def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    # keys = json.loads(output)
    return json.loads(output)

def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

def create_tx(coin, account, to, amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether") # convert 1.2 ETH to 120000000000 wei
        gasEstimate = w3.eth.estimateGas({ "to": to, "from": account, "amount": value })
        return {
            "to": to,
            "from": account,
            "value": value,
            "gas": gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce": w3.eth.getTransactionCount(account),
            "chainId": w3.eth.chain_id
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account.address, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}

pprint(coins)


JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [7]:
!pip install web3

     |████████████████████████████████| 487 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 1.8 MB/s ta 0:00:01
  Using cached eth_utils-1.10.0-py3-none-any.whl (24 kB)
  Using cached lru-dict-1.1.7.tar.gz (10 kB)
     |████████████████████████████████| 88 kB 947 kB/s eta 0:00:01
  Using cached ipfshttpclient-0.8.0a2-py3-none-any.whl (82 kB)
     |████████████████████████████████| 1.0 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 648 kB 2.3 MB/s eta 0:00:01
  Using cached hexbytes-0.2.2-py3-none-any.whl (6.1 kB)
  Using cached eth_hash-0.3.2-py3-none-any.whl (8.8 kB)
  Using cached eth_abi-2.1.1-py3-none-any.whl (27 kB)
  Using cached eth_typing-2.2.2-py3-none-any.whl (6.2 kB)
  Using cached multiaddr-0.0.9-py2.py3-none-any.whl (16 kB)
     |████████████████████████████████| 121 kB 2.5 MB/s eta 0:00:01
  Using cached async_timeout-3.0.1-py3-none-any.whl (8.2 kB)
     |████████████████████████████████| 45 kB 2.7 MB/s eta 0:00:01
  Using cached pa